In [3]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import os 

def webscraper(school, year):
    roster = f"https://www.sports-reference.com/cbb/schools/{school}/men/{year}.html#all_roster"
    result = requests.get(roster)
    content = result.text
    year_abr = year%2000 
    year_abr2 = year_abr - 1
    year_str = str(year_abr) + "_" + str(year_abr2)

    soup = BeautifulSoup(content, 'lxml')
    table = soup.find('table', class_= 'sortable stats_table')

    players_list = []

    # Iterate over each table row
    for row in table.find_all('tr')[1:]:  # Skip the first row (header row)
        columns1 = row.find_all('th')
        columns2 = row.find_all('td')

        player = columns1[0].text.strip()  # Player's name
        hometown = columns2[5].text.strip()  # Player's hometown
        high_school = columns2[6].text.strip()  # Player's high school
        
        players_list.append({
            'Player': player,
            'Hometown': hometown,
            'High School': high_school
        })

    df = pd.DataFrame(players_list)

    df[['City', 'State']] = df['Hometown'].str.extract(r'([A-Za-z\s]+),\s([A-Za-z.]+)')
    df.drop(columns=['Hometown'], inplace=True)

    df['High School'] = df['High School'].str.split(';').str[-1].str.strip()
    csv_folder = "csv_files"
    year_folder = year_str

    if not os.path.exists(csv_folder):
        os.makedirs(csv_folder)

    year_path = os.path.join(csv_folder, year_folder)
    if not os.path.exists(year_path):
        os.makedirs(year_path)

    df.to_csv(os.path.join(year_path, f'{school}_{year_str}.csv'), index=False)
    print(f'{school} csv uploaded!')
    return df


In [4]:
webscraper('illinois', 2024)

illinois csv uploaded!


,Player,High School,City,State
0,Terrence Shannon,Lincoln Park HS,Chicago,IL
1,Marcus Domask,Waupun HS,Waupun,WI
2,Coleman Hawkins,Prolific Prep,Sacramento,CA
3,Quincy Guerrier,Thetford Academy,Montreal,Canada
4,Ty Rodgers,Thornton Township (IL),Saginaw,MI
5,Dain Dainja,Park Center HS,Brooklyn Park,MN
6,Luke Goode,Homestead (IN),Fort Wayne,IN
7,Justin Harmon,Curie (IL),Chicago,IL
8,Dra Gibbs-Lawhorn,Word of God Christian Academy (NC),Lafayette,IN
9,Amani Hansberry,Mount Saint Joseph (MD),Baltimore,MD
